In [8]:
#pip install python-flint ppl

import numpy as np
import functools
from flint import fmpz_mat
from scipy.spatial import ConvexHull
import copy

In [2]:
print(fmpz_mat(2, 4, range(8)))

[0, 1, 2, 3]
[4, 5, 6, 7]


In [3]:
#source: https://github.com/LiamMcAllisterGroup/cytools/blob/main/src/cytools/utils.py

def gcd_float(a: float, b: float, tol: float = 1e-5) -> float:
    """
    **Description:**
    Compute the greatest common (floating-point) divisor of a and b. This is
    simply the largest floating point number that divides a and b. Uses the
    Euclidean algorithm.

    Warning - unexpected/buggy behavior can occur if b starts tiny. E.g.,
    gcd_float(100,0.1,0.2) returns 100.

    This only seems to be a risk if b *starts* below tol.

    **Arguments:**
    - `a`: The first number.
    - `b`: The second number.
    - `tol`: The tolerance for rounding.

    **Returns:**
    The gcd of a and b.

    **Example:**
    We compute the gcd of two floats. This function is useful since the
    standard gcd functions raise an error for non-integer values.
    ```python {2}
    from cytools.utils import gcd_float
    gcd_float(0.2, 0.5)
    # Should be 0.1, but there are small rounding errors
    # 0.09999999999999998
    ```
    """
    if abs(b) < tol: return abs(a)
    return gcd_float(b, a%b, tol)

# variant that computes gcd over all elements in arr
gcd_list = lambda arr: functools.reduce(gcd_float, arr)

In [4]:
#source: https://github.com/LiamMcAllisterGroup/cytools/blob/main/src/cytools/polytope.py
def poly_v_to_h(pts):
    """
    **Description:**
    Generate the H-representation of a polytope, given the V-representation.
    I.e., map points/vertices to hyperplanes inequalities.

    The o inequalities are in the form
        c_0 * x_0 + ... + c_{d-1} * x_{d-1} + c_d >= 0

    **Arguments:**
    - `pts`: The input points. Each row is a point.
    - `backend`: The backend to use. Currently, support "ppl", "qhull", and
        "palp".

    **Returns:**
    The hyperplane inequalities in the form
        c_0 * x_0 + ... + c_{d-1} * x_{d-1} + c_d >= 0
    and, depending on backend/dimension, the formal convex hull of the points.
    """
    # preliminary
    dim = len(pts[0])

    poly = ConvexHull(pts)

    # get the ineqs, ensure right sign and gcd
    ineqs = set()
    for eq in poly.equations:
        g = abs(gcd_list(eq))
        ineqs.add(tuple(-int(round(i/g)) for i in eq))
    ineqs = np.array(list(ineqs), dtype=int)

    return ineqs, poly

In [9]:
def normal_form(vertices) -> np.ndarray:
        """
        **Description:**
        Returns the normal form of the polytope as defined by Kreuzer-Skarke.

        **Arguments:**
        - `affine_transform`: Flag that determines whether to only use
            $SL^{\pm}(d,\mathbb{Z})$ transformations or also allow
            translations.
        - `backend`: Selects which backend to use. Options are "native", which
            uses native python code, or "palp", which uses PALP for the
            computation. There is a different convention for affine normal
            forms between the native algorithm and PALP, and PALP generally
            works better.

        **Returns:**
        The list of vertices in normal form.

        **Example:**
        We construct a polytope, and find its linear and affine normal forms.
        ```python {2,8}
        p = Polytope([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],[-1,-1,-6,-9]])
        p.normal_form()
        # array([[ 1,  0,  0,  0],
        #        [ 0,  1,  0,  0],
        #        [ 0,  0,  1,  0],
        #        [ 0,  0,  0,  1],
        #        [-9, -6, -1, -1]])
        p.normal_form(affine_transform=True)
        # array([[ 1,  0,  0,  0],
        #        [ 0,  1,  0,  0],
        #        [ 0,  0,  1,  0],
        #        [12, 17, 17, 18],
        #        [ 0,  0,  0,  0]])
        ```
        """
        # This function is based on code by Andrey Novoseltsev, Samuel Gonshaw,
        # Jan Keitel, and others, and is redistributed under the GNU General
        # Public License version 2+.
        #
        # The original code can be found at:
        # https://github.com/sagemath/sage/blob/develop/src/sage/geometry/lattice_polytope.py
        # https://trac.sagemath.org/ticket/13525

        # Define function that constructs permutation matrices
        def PGE(n, u, v):
            tmp_m = np.eye(n, dtype=int)
            if u == v:
                return tmp_m
            tmp_m[u-1,u-1] = 0
            tmp_m[v-1,v-1] = 0
            tmp_m[u-1,v-1] = 1
            tmp_m[v-1,u-1] = 1
            return tmp_m

        V = vertices
        pts_optimal = vertices
        out = poly_v_to_h(pts_optimal)
        _ineqs_optimal, _poly_optimal = out
        
        _dim_ambient = len(vertices[0])
        _dim = np.linalg.matrix_rank([list(pt)+[1] for pt in vertices]) - 1
        _dim_diff = _dim_ambient - _dim
    
        # convert to input representation
        shape_opt = _ineqs_optimal.shape
        shape = (shape_opt[0], shape_opt[1]+_dim_diff)

        _ineqs_input = np.zeros(shape, dtype=int)
        _ineqs_input[:, _dim_diff:] = _ineqs_optimal
        
        n_v = len(V)
        n_f = len(_ineqs_input)
        PM = np.array([n[:-1].dot(V.T) + n[-1] for n in _ineqs_input])
        n_s = 1
        prm = {0 : [np.eye(n_f, dtype=int), np.eye(n_v, dtype=int)]}
        for j in range(n_v):
            m = np.argmax([PM[0,prm[0][1].dot(range(n_v))][i] for i in range(j, n_v)])
            if m > 0:
                prm[0][1] = PGE(n_v, j+1, m+j+1).dot(prm[0][1])
        first_row = list(PM[0])
        # Arrange other rows one by one and compare with first row
        for k in range(1, n_f):
            # Error for k == 1 already!
            prm[n_s] = [np.eye(n_f, dtype=int), np.eye(n_v, dtype=int)]
            m = np.argmax(PM[:,prm[n_s][1].dot(range(n_v))][k])
            if m > 0:
                prm[n_s][1] = PGE(n_v, 1, m+1).dot(prm[n_s][1])
            d = PM[k,prm[n_s][1].dot(range(n_v))][0] - prm[0][1].dot(first_row)[0]
            if d < 0:
                # The largest elt of this row is smaller than largest elt
                # in 1st row, so nothing to do
                continue
            # otherwise:
            for i in range(1, n_v):
                m = np.argmax([PM[k,prm[n_s][1].dot(range(n_v))][j] for j in range(i, n_v)])
                if m > 0:
                    prm[n_s][1] = PGE(n_v, i+1, m+i+1).dot(prm[n_s][1])
                if d == 0:
                    d = PM[k,prm[n_s][1].dot(range(n_v))][i] - prm[0][1].dot(first_row)[i]
                    if d < 0:
                        break
            if d < 0:
                # This row is smaller than 1st row, so nothing to do
                del prm[n_s]
                continue
            prm[n_s][0] =  PGE(n_f, 1, k+1).dot(prm[n_s][0])
            if d == 0:
                # This row is the same, so we have a symmetry!
                n_s += 1
            else:
                # This row is larger, so it becomes the first row and
                # the symmetries reset.
                first_row = list(PM[k])
                prm = {0: prm[n_s]}
                n_s = 1
        prm = {k:prm[k] for k in prm if k < n_s}
        b = PM[prm[0][0].dot(range(n_f)),:][:,prm[0][1].dot(range(n_v))][0]
        # Work out the restrictions the current permutations
        # place on other permutations as a automorphisms
        # of the first row
        # The array is such that:
        # S = [i, 1, ..., 1 (ith), j, i+1, ..., i+1 (jth), k ... ]
        # describes the "symmetry blocks"
        S = list(range(1, n_v+1))
        for i in range(1, n_v):
            if b[i-1] == b[i]:
                S[i] = S[i-1]
                S[S[i]-1] += 1
            else:
                S[i] = i + 1
        # We determine the other rows of PM_max in turn by use of perms and
        # aut on previous rows.
        for l in range(1, n_f-1):
            n_s = len(prm)
            n_s_bar = n_s
            cf = 0
            l_r = [0]*n_v
            # Search for possible local permutations based off previous
            # global permutations.
            for k in range(n_s_bar-1, -1, -1):
                # number of local permutations associated with current global
                n_p = 0
                ccf = cf
                prmb = {0: copy.copy(prm[k])}
                # We look for the line with the maximal entry in the first
                # subsymmetry block, i.e. we are allowed to swap elements
                # between 0 and S(0)
                for s in range(l, n_f):
                    for j in range(1, S[0]):
                        v = PM[prmb[n_p][0].dot(range(n_f)),:][:,prmb[n_p][1].dot(range(n_v))][s]
                        if v[0] < v[j]:
                            prmb[n_p][1] = PGE(n_v, 1, j+1).dot(prmb[n_p][1])
                    if ccf == 0:
                        l_r[0] = PM[prmb[n_p][0].dot(range(n_f)),:][:,prmb[n_p][1].dot(range(n_v))][s,0]
                        prmb[n_p][0] = PGE(n_f, l+1, s+1).dot(prmb[n_p][0])
                        n_p += 1
                        ccf = 1
                        prmb[n_p] = copy.copy(prm[k])
                    else:
                        d1 = PM[prmb[n_p][0].dot(range(n_f)),:][:,prmb[n_p][1].dot(range(n_v))][s,0]
                        d = d1 - l_r[0]
                        if d < 0:
                            # We move to the next line
                            continue
                        if d == 0:
                            # Maximal values agree, so possible symmetry
                            prmb[n_p][0] = PGE(n_f, l+1, s+1).dot(prmb[n_p][0])
                            n_p += 1
                            prmb[n_p] = copy.copy(prm[k])
                        else:
                            # We found a greater maximal value for first entry.
                            # It becomes our new reference:
                            l_r[0] = d1
                            prmb[n_p][0] = PGE(n_f, l+1, s+1).dot(prmb[n_p][0])
                            # Forget previous work done
                            cf = 0
                            prmb = {0:copy.copy(prmb[n_p])}
                            n_p = 1
                            prmb[n_p] = copy.copy(prm[k])
                            n_s = k + 1
                # Check if the permutations found just now work
                # with other elements
                for c in range(1, n_v):
                    h = S[c]
                    ccf = cf
                    # Now let us find out where the end of the
                    # next symmetry block is:
                    if  h < c+1:
                        h = S[h-1]
                    s = n_p
                    # Check through this block for each possible permutation
                    while s > 0:
                        s -= 1
                        # Find the largest value in this symmetry block
                        for j in range(c+1, h):
                            v = PM[prmb[s][0].dot(range(n_f)),:][:,prmb[s][1].dot(range(n_v))][l]
                            if v[c] < v[j]:
                                prmb[s][1] = PGE(n_v, c+1, j+1).dot(prmb[s][1])
                        if ccf == 0:
                            # Set reference and carry on to next permutation
                            l_r[c] = PM[prmb[s][0].dot(range(n_f)),:][:,prmb[s][1].dot(range(n_v))][l,c]
                            ccf = 1
                        else:
                            d1 = PM[prmb[s][0].dot(range(n_f)),:][:,prmb[s][1].dot(range(n_v))][l,c]
                            d = d1 - l_r[c]
                            if d < 0:
                                n_p -= 1
                                if s < n_p:
                                    prmb[s] = copy.copy(prmb[n_p])
                            elif d > 0:
                                # The current case leads to a smaller matrix,
                                # hence this case becomes our new reference
                                l_r[c] = d1
                                cf = 0
                                n_p = s + 1
                                n_s = k + 1
                # Update permutations
                if n_s-1 > k:
                    prm[k] = copy.copy(prm[n_s-1])
                n_s -= 1
                for s in range(n_p):
                    prm[n_s] = copy.copy(prmb[s])
                    n_s += 1
                cf = n_s
            prm = {k:prm[k] for k in prm if k < n_s}
            # If the automorphisms are not already completely restricted,
            # update them
            if S != list(range(1, n_v+1)):
                # Take the old automorphisms and update by
                # the restrictions the last worked out
                # row imposes.
                c = 0
                M = PM[prm[0][0].dot(range(n_f)),:][:,prm[0][1].dot(range(n_v))][l]
                while c < n_v:
                    s = S[c] + 1
                    S[c] = c + 1
                    c += 1
                    while c < s-1:
                        if M[c] == M[c-1]:
                            S[c] = S[c-1]
                            S[S[c]-1] += 1
                        else:
                            S[c] = c + 1
                        c += 1
        # Now we have the perms, we construct PM_max using one of them
        PM_max = PM[prm[0][0].dot(range(n_f)),:][:,prm[0][1].dot(range(n_v))]
        
        # Finally arrange the points the the canonical order
        p_c = np.eye(n_v, dtype=int)
        M_max = [max([PM_max[i][j] for i in range(n_f)]) for j in range(n_v)]
        S_max = [sum([PM_max[i][j] for i in range(n_f)]) for j in range(n_v)]
        for i in range(n_v):
            k = i
            for j in range(i+1, n_v):
                if M_max[j] < M_max[k] or (M_max[j] == M_max[k] and S_max[j] < S_max[k]):
                    k = j
            if not k == i:
                M_max[i], M_max[k] = M_max[k], M_max[i]
                S_max[i], S_max[k] = S_max[k], S_max[i]
                p_c = PGE(n_v, 1+i, 1+k).dot(p_c)
        # Create array of possible NFs.
        prm = [p_c.dot(l[1]) for l in prm.values()]
        Vs = [np.array(fmpz_mat(V.T[:,sig.dot(range(n_v))].tolist()).hnf().tolist(), dtype=int).tolist() for sig in prm]
        Vmin = min(Vs)
        
        return np.array(np.array(Vmin).T)

In [11]:
vertices = np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],[-1,-1,-6,-9]])
print(vertices)
print(normal_form(vertices))

[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0  1]
 [-1 -1 -6 -9]]
[[ 1  0  0  0]
 [ 0  1  0  0]
 [ 0  0  1  0]
 [ 0  0  0  1]
 [-9 -6 -1 -1]]
